In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)   # 去掉第一层索引
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

# zero = sales_train[sales_train.date_block_num==0]
# ic(zero.shop_id.unique(), len(zero.item_id.unique()), len(zero.shop_id.unique()), len(zero.shop_id.unique()) * len(zero.item_id.unique()))
# ic(sales_train.shop_id.unique(), len(sales_train.item_id.unique()), len(sales_train.shop_id.unique()), len(sales_train.shop_id.unique()) * len(sales_train.item_id.unique()))

"""组合date_block_num,shop_id,item_id(部分) 总量：10913850"""
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

# 分组
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

# test数据
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

# 合并matrix,test
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix['item_cnt_month'].fillna(0, inplace=True)

# 商品信息
items = pd.read_csv('./data/items.csv')
items = items[['item_id', 'item_category_id']]
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并

# 商品类别
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并

# 商店信息
shops = pd.read_csv('./data/shops.csv')
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])

def st(name):
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt
shops['shop_type'] = shops['shop_name'].apply(st)

shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)


"""历史信息"""

def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)



# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商品大类） ++++++++++++ 和 月销量（商品）是重复的，因为每一个商品，类别是确定的，大类也是确定的
group = matrix.groupby(['date_block_num', 'item_id', 'cat_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_type_avg_item_cnt')
matrix.drop('date_item_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)



# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)] = (matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

# 月销量（商店类型）
group = matrix.groupby(['date_block_num','shop_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shoptype_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shoptype_avg_item_cnt')
matrix.drop('date_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店类型）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_shoptype_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_shoptype_avg_item_cnt')
matrix.drop('date_item_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店-商品）
group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shopitem_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shopitem_avg_item_cnt')
matrix.drop('date_shopitem_avg_item_cnt', axis=1, inplace=True)

# 趋势特征
group = matrix.groupby('item_id').agg({'item_cnt_month': 'mean'})
group.columns = ['trend_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['trend_date_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1, 2, 3, 4, 5, 6, 12]
matrix = lag_features(matrix, lags, 'trend_date_item_avg_cnt_month')
for i in lags:
    matrix['delta_cnt_month_lag_' + str(i)] = (matrix['trend_date_item_avg_cnt_month_lag_' + str(i)] - matrix[
        'trend_item_avg_cnt_month']) / matrix['trend_item_avg_cnt_month']


def select_trend2(row):
    for i in lags:
        if pd.notnull(row['delta_cnt_month_lag_' + str(i)]):  # 如果不是NaN
            return row['delta_cnt_month_lag_' + str(i)]
    return 0  # 如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势


matrix['delta_cnt_month_lag'] = matrix.apply(select_trend2, axis=1)

matrix['delta_cnt_month_lag'] = matrix['delta_cnt_month_lag'].astype(np.float16)

features_to_drop = ['trend_item_avg_cnt_month', 'trend_date_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['trend_date_item_avg_cnt_month_lag_' + str(i)]
    features_to_drop += ['delta_cnt_month_lag_' + str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)


# 趋势特征 +++++++++++++++++++++++++++++++++++++
group = matrix.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_shop_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['shop_id', 'item_id'], how='left')

group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_date_shop_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'qushi_date_shop_item_avg_cnt_month')
for i in lags:
    matrix['delta2_cnt_month_lag_'+str(i)] = (matrix['qushi_date_shop_item_avg_cnt_month_lag_' + str(i)] - matrix['qushi_shop_item_avg_cnt_month']) / matrix['qushi_shop_item_avg_cnt_month']

def select_trend3(row):
    for i in lags:
        if pd.notnull(row['delta2_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta2_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta2_cnt_month_lag'] = matrix.apply(select_trend3, axis=1)
matrix['delta2_cnt_month_lag'] = matrix['delta2_cnt_month_lag'].astype(np.float16)

features_to_drop = ['qushi_shop_item_avg_cnt_month','qushi_date_shop_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['delta2_cnt_month_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

# 找到有NaN值的列，然后把那些列中的NaN值填充0
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)

for i in column_null:
    matrix[i].fillna(0, inplace=True)

In [2]:
matrix.shape

(6639294, 88)

In [3]:
# matrix_88_bak = matrix.copy()

In [10]:
matrix.drop(['qushi_date_shop_item_avg_cnt_month_lag_1',
       'qushi_date_shop_item_avg_cnt_month_lag_2',
       'qushi_date_shop_item_avg_cnt_month_lag_3',
       'qushi_date_shop_item_avg_cnt_month_lag_4',
       'qushi_date_shop_item_avg_cnt_month_lag_5',
       'qushi_date_shop_item_avg_cnt_month_lag_6',
       'qushi_date_shop_item_avg_cnt_month_lag_12', 'delta2_cnt_month_lag'], axis=1, inplace=True)

In [11]:
matrix.shape

(6639294, 80)

In [12]:
# matrix_80_bak = matrix.copy()

In [13]:
"""建模"""

trainData = matrix[matrix['date_block_num'] < 33]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 5000,
    'max_depth': 8,
    'num_leaves': 180,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.005,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

d:\program files\python37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.264886 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12987
[LightGBM] [Info] Number of data points in the train set: 6425094, number of used features: 79
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.287729
[1]	training's rmse: 1.18398	valid_1's rmse: 1.13485
[2]	training's rmse: 1.18154	valid_1's rmse: 1.13301
[3]	training's rmse: 1.17923	valid_1's rmse: 1.13122
[4]	training's rmse: 1.17678	valid_1's rmse: 1.12943
[5]	training's rmse: 1.17424	valid_1's rmse: 1.12742
[6]	training's rmse: 1.17188	valid_1's rmse: 1.12572
[7]	training's rmse: 1.16998	valid_1's rmse: 1.1243
[8]	training's rmse: 1.16757	valid_1's rmse:

[141]	training's rmse: 0.968481	valid_1's rmse: 0.980188
[142]	training's rmse: 0.967551	valid_1's rmse: 0.979672
[143]	training's rmse: 0.966557	valid_1's rmse: 0.979027
[144]	training's rmse: 0.965626	valid_1's rmse: 0.97839
[145]	training's rmse: 0.96469	valid_1's rmse: 0.977796
[146]	training's rmse: 0.963859	valid_1's rmse: 0.977228
[147]	training's rmse: 0.963017	valid_1's rmse: 0.976715
[148]	training's rmse: 0.962199	valid_1's rmse: 0.976206
[149]	training's rmse: 0.961361	valid_1's rmse: 0.975654
[150]	training's rmse: 0.960564	valid_1's rmse: 0.975158
[151]	training's rmse: 0.959721	valid_1's rmse: 0.974629
[152]	training's rmse: 0.958886	valid_1's rmse: 0.974067
[153]	training's rmse: 0.958087	valid_1's rmse: 0.973583
[154]	training's rmse: 0.957068	valid_1's rmse: 0.972783
[155]	training's rmse: 0.956169	valid_1's rmse: 0.972152
[156]	training's rmse: 0.955329	valid_1's rmse: 0.971597
[157]	training's rmse: 0.954338	valid_1's rmse: 0.970979
[158]	training's rmse: 0.953344	v

[286]	training's rmse: 0.881425	valid_1's rmse: 0.923979
[287]	training's rmse: 0.881127	valid_1's rmse: 0.923817
[288]	training's rmse: 0.880803	valid_1's rmse: 0.923657
[289]	training's rmse: 0.880479	valid_1's rmse: 0.923497
[290]	training's rmse: 0.880035	valid_1's rmse: 0.923101
[291]	training's rmse: 0.879731	valid_1's rmse: 0.922951
[292]	training's rmse: 0.879314	valid_1's rmse: 0.922697
[293]	training's rmse: 0.879017	valid_1's rmse: 0.922552
[294]	training's rmse: 0.878741	valid_1's rmse: 0.92238
[295]	training's rmse: 0.878456	valid_1's rmse: 0.922236
[296]	training's rmse: 0.878148	valid_1's rmse: 0.922087
[297]	training's rmse: 0.877678	valid_1's rmse: 0.921786
[298]	training's rmse: 0.877327	valid_1's rmse: 0.921613
[299]	training's rmse: 0.877056	valid_1's rmse: 0.921408
[300]	training's rmse: 0.87671	valid_1's rmse: 0.921202
[301]	training's rmse: 0.876324	valid_1's rmse: 0.920982
[302]	training's rmse: 0.875916	valid_1's rmse: 0.920691
[303]	training's rmse: 0.875683	v

[431]	training's rmse: 0.845337	valid_1's rmse: 0.90259
[432]	training's rmse: 0.845233	valid_1's rmse: 0.902516
[433]	training's rmse: 0.845061	valid_1's rmse: 0.9024
[434]	training's rmse: 0.844776	valid_1's rmse: 0.902251
[435]	training's rmse: 0.844604	valid_1's rmse: 0.90218
[436]	training's rmse: 0.844468	valid_1's rmse: 0.902118
[437]	training's rmse: 0.844317	valid_1's rmse: 0.902072
[438]	training's rmse: 0.844166	valid_1's rmse: 0.901976
[439]	training's rmse: 0.843974	valid_1's rmse: 0.901896
[440]	training's rmse: 0.843817	valid_1's rmse: 0.901819
[441]	training's rmse: 0.843687	valid_1's rmse: 0.901768
[442]	training's rmse: 0.843575	valid_1's rmse: 0.901712
[443]	training's rmse: 0.843399	valid_1's rmse: 0.901625
[444]	training's rmse: 0.843254	valid_1's rmse: 0.901552
[445]	training's rmse: 0.843133	valid_1's rmse: 0.901438
[446]	training's rmse: 0.842999	valid_1's rmse: 0.901375
[447]	training's rmse: 0.842865	valid_1's rmse: 0.9013
[448]	training's rmse: 0.842697	valid

...

[4271]	training's rmse: 0.738879	valid_1's rmse: 0.825501
[4272]	training's rmse: 0.738868	valid_1's rmse: 0.825495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4273]	training's rmse: 0.738862	valid_1's rmse: 0.825492
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4274]	training's rmse: 0.738854	valid_1's rmse: 0.82549
[4275]	training's rmse: 0.738847	valid_1's rmse: 0.825488
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4276]	training's rmse: 0.738842	valid_1's rmse: 0.825486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4277]	training's rmse: 0.738836	valid_1's rmse: 0.825483
[4278]	training's rmse: 0.738822	valid_1's rmse: 0.825471
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4279]	training's rmse: 0.73881	valid_1's rmse: 0.825452
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4280]	training's rmse: 0.7388

[4350]	training's rmse: 0.738131	valid_1's rmse: 0.824891
[4351]	training's rmse: 0.738124	valid_1's rmse: 0.824874
[4352]	training's rmse: 0.738096	valid_1's rmse: 0.824861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4353]	training's rmse: 0.738079	valid_1's rmse: 0.824837
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4354]	training's rmse: 0.738074	valid_1's rmse: 0.824833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4355]	training's rmse: 0.73807	valid_1's rmse: 0.824832
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4356]	training's rmse: 0.738066	valid_1's rmse: 0.824831
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4357]	training's rmse: 0.73806	valid_1's rmse: 0.82482
[4358]	training's rmse: 0.738052	valid_1's rmse: 0.824817
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4359]	training's rmse: 0.73804

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4427]	training's rmse: 0.737453	valid_1's rmse: 0.824297
[4428]	training's rmse: 0.737443	valid_1's rmse: 0.824282
[4429]	training's rmse: 0.737431	valid_1's rmse: 0.824278
[4430]	training's rmse: 0.737387	valid_1's rmse: 0.824265
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4431]	training's rmse: 0.737384	valid_1's rmse: 0.824264
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4432]	training's rmse: 0.737379	valid_1's rmse: 0.824263
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4433]	training's rmse: 0.737371	valid_1's rmse: 0.824258
[4434]	training's rmse: 0.737351	valid_1's rmse: 0.824255
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4435]	training's rmse: 0.737344	valid_1's rmse: 0.824253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4436]	training's rmse: 0.73

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4502]	training's rmse: 0.736793	valid_1's rmse: 0.823772
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4503]	training's rmse: 0.736789	valid_1's rmse: 0.823771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4504]	training's rmse: 0.736782	valid_1's rmse: 0.823767
[4505]	training's rmse: 0.736776	valid_1's rmse: 0.823764
[4506]	training's rmse: 0.736768	valid_1's rmse: 0.82375
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4507]	training's rmse: 0.736763	valid_1's rmse: 0.823746
[4508]	training's rmse: 0.736752	valid_1's rmse: 0.823737
[4509]	training's rmse: 0.736742	valid_1's rmse: 0.823727
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4510]	training's rmse: 0.736737	valid_1's rmse: 0.823725
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4511]	training's rmse: 0.736

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4581]	training's rmse: 0.736122	valid_1's rmse: 0.823091
[4582]	training's rmse: 0.736083	valid_1's rmse: 0.823032
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4583]	training's rmse: 0.736079	valid_1's rmse: 0.823028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4584]	training's rmse: 0.736074	valid_1's rmse: 0.823024
[4585]	training's rmse: 0.736061	valid_1's rmse: 0.82299
[4586]	training's rmse: 0.736051	valid_1's rmse: 0.822984
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4587]	training's rmse: 0.736048	valid_1's rmse: 0.822982
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4588]	training's rmse: 0.736042	valid_1's rmse: 0.822979
[4589]	training's rmse: 0.736035	valid_1's rmse: 0.822976
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4590]	training's rmse: 0.736

# 再训练一次0-33个月的数据

In [17]:
trainData = matrix[matrix['date_block_num'] < 34]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

# validData = matrix[matrix['date_block_num'] == 34]
# label_valid = validData['item_cnt_month']
# X_valid = validData.drop('item_cnt_month', axis=1)

In [18]:
train_data = lgb.Dataset(data=X_train, label=label_train)
# valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 2000,
    'max_depth': 8,
    'num_leaves': 180,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.005,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data])

d:\program files\python37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.242918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12987
[LightGBM] [Info] Number of data points in the train set: 6425094, number of used features: 79
[LightGBM] [Info] Start training from score 0.287729
[1]	training's rmse: 1.18398
[2]	training's rmse: 1.18154
[3]	training's rmse: 1.17923
[4]	training's rmse: 1.17678
[5]	training's rmse: 1.17424
[6]	training's rmse: 1.17188
[7]	training's rmse: 1.16998
[8]	training's rmse: 1.16757
[9]	training's rmse: 1.16538
[10]	training's rmse: 1.16344
[11]	training's rmse: 1.16121
[12]	training's rmse: 1.15918
[13]	training's rmse: 1.15699
[14]	training's rmse: 1.15474
[15]	training's rmse: 1.15254
[16]	training's rmse: 1.1504
[1

[249]	training's rmse: 0.89689
[250]	training's rmse: 0.89644
[251]	training's rmse: 0.895986
[252]	training's rmse: 0.895556
[253]	training's rmse: 0.895091
[254]	training's rmse: 0.894672
[255]	training's rmse: 0.894223
[256]	training's rmse: 0.893826
[257]	training's rmse: 0.89337
[258]	training's rmse: 0.89294
[259]	training's rmse: 0.892456
[260]	training's rmse: 0.892004
[261]	training's rmse: 0.891638
[262]	training's rmse: 0.89119
[263]	training's rmse: 0.890893
[264]	training's rmse: 0.890554
[265]	training's rmse: 0.890076
[266]	training's rmse: 0.889622
[267]	training's rmse: 0.889139
[268]	training's rmse: 0.888767
[269]	training's rmse: 0.888337
[270]	training's rmse: 0.887984
[271]	training's rmse: 0.887639
[272]	training's rmse: 0.887188
[273]	training's rmse: 0.886831
[274]	training's rmse: 0.886383
[275]	training's rmse: 0.885966
[276]	training's rmse: 0.885481
[277]	training's rmse: 0.885069
[278]	training's rmse: 0.884571
[279]	training's rmse: 0.884145
[280]	trainin

[506]	training's rmse: 0.834287
[507]	training's rmse: 0.834176
[508]	training's rmse: 0.834045
[509]	training's rmse: 0.833981
[510]	training's rmse: 0.833872
[511]	training's rmse: 0.833725
[512]	training's rmse: 0.833631
[513]	training's rmse: 0.833534
[514]	training's rmse: 0.833455
[515]	training's rmse: 0.833358
[516]	training's rmse: 0.833275
[517]	training's rmse: 0.833065
[518]	training's rmse: 0.832943
[519]	training's rmse: 0.832682
[520]	training's rmse: 0.832564
[521]	training's rmse: 0.832489
[522]	training's rmse: 0.832303
[523]	training's rmse: 0.832068
[524]	training's rmse: 0.831962
[525]	training's rmse: 0.831864
[526]	training's rmse: 0.831642
[527]	training's rmse: 0.831523
[528]	training's rmse: 0.83133
[529]	training's rmse: 0.8312
[530]	training's rmse: 0.831029
[531]	training's rmse: 0.830897
[532]	training's rmse: 0.830807
[533]	training's rmse: 0.830674
[534]	training's rmse: 0.830577
[535]	training's rmse: 0.830436
[536]	training's rmse: 0.830231
[537]	train

[749]	training's rmse: 0.810029
[750]	training's rmse: 0.809955
[751]	training's rmse: 0.809858
[752]	training's rmse: 0.809746
[753]	training's rmse: 0.809688
[754]	training's rmse: 0.809565
[755]	training's rmse: 0.809526
[756]	training's rmse: 0.809437
[757]	training's rmse: 0.80939
[758]	training's rmse: 0.809344
[759]	training's rmse: 0.80928
[760]	training's rmse: 0.809208
[761]	training's rmse: 0.80907
[762]	training's rmse: 0.809021
[763]	training's rmse: 0.808923
[764]	training's rmse: 0.808877
[765]	training's rmse: 0.808788
[766]	training's rmse: 0.808722
[767]	training's rmse: 0.808668
[768]	training's rmse: 0.808552
[769]	training's rmse: 0.808361
[770]	training's rmse: 0.808317
[771]	training's rmse: 0.808274
[772]	training's rmse: 0.808227
[773]	training's rmse: 0.80816
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[774]	training's rmse: 0.808113
[775]	training's rmse: 0.808057
[776]	training's rmse: 0.807978
[LightGBM] [Warning] No further s

...

[1199]	training's rmse: 0.786608
[1200]	training's rmse: 0.786583
[1201]	training's rmse: 0.786548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1202]	training's rmse: 0.786526
[1203]	training's rmse: 0.786505
[1204]	training's rmse: 0.78648
[1205]	training's rmse: 0.786455
[1206]	training's rmse: 0.7864
[1207]	training's rmse: 0.786368
[1208]	training's rmse: 0.786328
[1209]	training's rmse: 0.786298
[1210]	training's rmse: 0.786275
[1211]	training's rmse: 0.786248
[1212]	training's rmse: 0.786212
[1213]	training's rmse: 0.786182
[1214]	training's rmse: 0.786153
[1215]	training's rmse: 0.786114
[1216]	training's rmse: 0.786097
[1217]	training's rmse: 0.786065
[1218]	training's rmse: 0.786032
[1219]	training's rmse: 0.786001
[1220]	training's rmse: 0.785969
[1221]	training's rmse: 0.785922
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1222]	training's rmse: 0.785896
[1223]	training's rmse: 0.785859
[1224]	training's rmse: 0.78

[1392]	training's rmse: 0.780256
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1393]	training's rmse: 0.780236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1394]	training's rmse: 0.780216
[1395]	training's rmse: 0.780173
[1396]	training's rmse: 0.780147
[1397]	training's rmse: 0.780121
[1398]	training's rmse: 0.780079
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1399]	training's rmse: 0.780064
[1400]	training's rmse: 0.780041
[1401]	training's rmse: 0.779995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1402]	training's rmse: 0.779977
[1403]	training's rmse: 0.779945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1404]	training's rmse: 0.779911
[1405]	training's rmse: 0.779881
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1406]	training's rmse: 0.779861
[1407]	training's rmse: 0.779819
[LightGBM] [Warning] N

[1566]	training's rmse: 0.775333
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1567]	training's rmse: 0.775318
[1568]	training's rmse: 0.775296
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1569]	training's rmse: 0.77528
[1570]	training's rmse: 0.775244
[1571]	training's rmse: 0.775216
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1572]	training's rmse: 0.7752
[1573]	training's rmse: 0.77515
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1574]	training's rmse: 0.77513
[1575]	training's rmse: 0.775112
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1576]	training's rmse: 0.775095
[1577]	training's rmse: 0.775074
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1578]	training's rmse: 0.775044
[1579]	training's rmse: 0.775031
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1580]	training's 

[1738]	training's rmse: 0.771406
[1739]	training's rmse: 0.771387
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1740]	training's rmse: 0.771375
[1741]	training's rmse: 0.771358
[1742]	training's rmse: 0.771343
[1743]	training's rmse: 0.771314
[1744]	training's rmse: 0.771295
[1745]	training's rmse: 0.771278
[1746]	training's rmse: 0.771262
[1747]	training's rmse: 0.771248
[1748]	training's rmse: 0.771214
[1749]	training's rmse: 0.771195
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1750]	training's rmse: 0.771181
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1751]	training's rmse: 0.771164
[1752]	training's rmse: 0.771136
[1753]	training's rmse: 0.771119
[1754]	training's rmse: 0.771102
[1755]	training's rmse: 0.771074
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1756]	training's rmse: 0.771056
[1757]	training's rmse: 0.771038
[LightGBM] [Warning] No further splits w

[1896]	training's rmse: 0.768345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1897]	training's rmse: 0.76833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1898]	training's rmse: 0.768302
[1899]	training's rmse: 0.768285
[1900]	training's rmse: 0.768271
[1901]	training's rmse: 0.768245
[1902]	training's rmse: 0.768225
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1903]	training's rmse: 0.768211
[1904]	training's rmse: 0.768192
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1905]	training's rmse: 0.768177
[1906]	training's rmse: 0.768152
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1907]	training's rmse: 0.76814
[1908]	training's rmse: 0.768125
[1909]	training's rmse: 0.768109
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1910]	training's rmse: 0.768089
[1911]	training's rmse: 0.768063
[1912]	training's rmse: 

In [19]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})

test0 = test[test.item_id.isin(six_zero_item_id)]
ids = list(test0.ID.values)
submission.loc[submission.ID.isin(ids), 'item_cnt_month'] = 0.0
submission.to_csv('./submit/sub6.csv', index=False)